### Установка зависимостей

In [ ]:
# # Базовое обновление инструментов
# %pip install -U pip setuptools wheel

# # PyTorch
# %pip install -U torch torchvision torchaudio

# # Установщик OpenMMLab
# %pip install -U openmim

# # MMEngine и MMCV 2.1.x
# !mim install "mmengine>=0.10.0"
# !mim install "mmcv==2.1.0"

# # MMDetection для авто-детекции людей 3.2.x
# !mim install "mmdet==3.2.0"

# # MMPose 1.x
# !mim install "mmpose==1.3.2"

In [ ]:
# %pip install -U gdown imageio-ffmpeg --quiet
# %pip install -q -U imageio imageio-ffmpeg

# %pip install -U "numpy==2.2.6" cython

# # снесём и пересоберём оба COCO-пакета
# %pip uninstall -y xtcocotools pycocotools
# %pip install --no-binary=xtcocotools,pycocotools --no-cache-dir xtcocotools pycocotools

# # на всякий случай переустановим mmpose (поверх тех же версий)
# %pip install -U --no-cache-dir "mmpose==1.3.2"

### Перезапускаем ядро, проверяем версии

In [ ]:
import importlib
import platform
import sys

import numpy as np
import torch
import xtcocotools._mask as _mask
from mmpose.apis import MMPoseInferencer


def _ver(pkg):
    try:
        m = importlib.import_module(pkg)
        return getattr(m, "__version__", "N/A")
    except Exception as e:
        return f"not installed ({e})"


print("Platform:", platform.platform())
print("Python:", sys.version)
print("\ntorch:", _ver("torch"))
print("torchvision:", _ver("torchvision"))
print("mmengine:", _ver("mmengine"))
print("mmcv:", _ver("mmcv"))
print("mmdet:", _ver("mmdet"))
print("\nMMPoseInferencer import OK")
print("mmpose:", _ver("mmpose"))
print("\nxtcocotools with numpy version:", _ver("numpy"))


print(
    "\nDevice selected:", "mps" if torch.backends.mps.is_available() else "cpu"
)

Platform: macOS-15.6.1-arm64-arm-64bit
Python: 3.11.13 (main, Jun  3 2025, 18:38:25) [Clang 17.0.0 (clang-1700.0.13.3)]

torch: 2.2.2
torchvision: 0.17.2
mmengine: 0.10.7
mmcv: 2.1.0
mmdet: 3.2.0

MMPoseInferencer import OK
mmpose: 1.3.2

xtcocotools with numpy version: 1.26.4

Device selected: mps


### Каталоги проекта

In [ ]:
from pathlib import Path

BASE_DIR = Path(".").resolve()
INPUT_DIR = BASE_DIR / "input_compare_angles"
OUTPUT_DIR = BASE_DIR / "output_compare_angles"
WEIGHTS_DIR = BASE_DIR / "weights"
INPUT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
WEIGHTS_DIR.mkdir(parents=True, exist_ok=True)

print("BASE_DIR =", BASE_DIR)
print("INPUT_DIR =", INPUT_DIR)
print("OUTPUT_DIR =", OUTPUT_DIR)
print("WEIGHTS_DIR =", WEIGHTS_DIR)

BASE_DIR = /Users/pussykiller/Учеба/Семестр 3/Проектный практикум/CV-Workout-Tracker/notebooks/mmpose
INPUT_DIR = /Users/pussykiller/Учеба/Семестр 3/Проектный практикум/CV-Workout-Tracker/notebooks/mmpose/input_compare_angles
OUTPUT_DIR = /Users/pussykiller/Учеба/Семестр 3/Проектный практикум/CV-Workout-Tracker/notebooks/mmpose/output_compare_angles
WEIGHTS_DIR = /Users/pussykiller/Учеба/Семестр 3/Проектный практикум/CV-Workout-Tracker/notebooks/mmpose/weights


# Попытка на фото

### 1. Скачиваем изображение

In [ ]:
from pathlib import Path

import gdown
from IPython.display import display
from PIL import Image

GD_URL = "https://drive.google.com/file/d/1RenekFPFYrB1UhAHhdZf67LeCUw8WbZh/view?usp=sharing"
img_path = INPUT_DIR / "cheliki_na_turnike.jpg"

gdown.download(GD_URL, str(img_path), quiet=False, fuzzy=True)
assert img_path.exists(), f"Изображение не скачалось: {img_path}"

print("Скачано в:", img_path)
print("Размер: {:.2f} МБ".format(img_path.stat().st_size / (1024**2)))

display(Image.open(img_path))

### 2. Применяем MMPoseInferencer на скачанном изображении

In [ ]:
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
from mmpose.apis import MMPoseInferencer

device = "cpu"

try:
    torch.serialization.add_safe_globals(
        [
            np.core.multiarray._reconstruct,
            np.dtype,
            np.ufunc,
        ]
    )
except Exception:
    pass

print("Используем изображение:", img_path)

# Инициализируем унифицированный инференсер MMPose
# с alias "human" (детектор + оценка позы)
inferencer = MMPoseInferencer("human", device=device)

result_gen = inferencer(
    str(img_path),
    return_vis=True,  # вернуть массив визуализации в result['visualization']
    show=False,  # не открывать отдельное окно
    radius=16,
    thickness=8,  # сделать точки/скелет чуть заметнее
    vis_out_dir=str(
        OUTPUT_DIR / "vis"
    ),  # сюда сохранятся картинки с разметкой
    pred_out_dir=str(OUTPUT_DIR / "pred"),  # сюда — JSON с ключевыми точками
)

result = next(result_gen)

vis_list = result.get("visualization", [])
if not vis_list:
    raise RuntimeError(
        "Нет визуализации в результате. Убедитесь, что return_vis=True."
    )

vis_img = vis_list[
    0
]  # это RGB-изображение с нарисованными ключевыми точками/скелетом

# 5) Сохраним и покажем
out_file = OUTPUT_DIR / f"{img_path.stem}_pose_vis.jpg"
# cv2.imwrite ожидает BGR -> конвертируем
cv2.imwrite(str(out_file), vis_img[..., ::-1])

print("Визуализация сохранена в:", out_file)

plt.figure(figsize=(10, 8))
plt.imshow(vis_img)  # vis_img уже в RGB
plt.axis("off")
plt.title("MMPose: скелетики на изображении")
plt.show()

# Попытка на видео

### 1. Скачиваем видео и анализируем

In [ ]:
import os
from pathlib import Path

import cv2
import gdown

GD_URL = "https://drive.google.com/file/d/1iDHBbYgV_sYRyVUi_BLdqcGogD_CDXmB/view?usp=sharing"
video_pth = INPUT_DIR / "tiktonik_360p.mp4"

video_pth.unlink(missing_ok=True)
gdown.download(GD_URL, str(video_pth), quiet=False, fuzzy=True)
assert video_pth.exists(), f"Видео не скачалось: {video_pth}"


cap = cv2.VideoCapture(str(video_pth))
if not cap.isOpened():
    raise RuntimeError(f"Не удалось открыть видео: {video_pth}")

fps = cap.get(cv2.CAP_PROP_FPS) or 25.0
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) or 0)
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) or 0)
n = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
cap.release()

print(f"Видео: {video_pth}")
print(f"Размер: {w}x{h}, FPS: {fps:.3f}, кадров: {n}")

Downloading...
From: https://drive.google.com/uc?id=1iDHBbYgV_sYRyVUi_BLdqcGogD_CDXmB
To: /Users/pussykiller/Учеба/Семестр 3/Проектный практикум/CV-Workout-Tracker/notebooks/mmpose/input_compare_angles/tiktonik_360p.mp4
100%|██████████| 2.30M/2.30M [00:00<00:00, 3.54MB/s]

Видео: /Users/pussykiller/Учеба/Семестр 3/Проектный практикум/CV-Workout-Tracker/notebooks/mmpose/input_compare_angles/tiktonik_360p.mp4
Размер: 482x360, FPS: 30.000, кадров: 1185


### 2. Применяем MMPoseInferencer на скачанном видео

In [ ]:
import os
from pathlib import Path

import cv2
import imageio
import numpy as np
import torch
from mmpose.apis import MMPoseInferencer

# Пути и устройство
device = "cpu"  # на macOS так избегаем проблемы NMS на MPS
out_dir = OUTPUT_DIR / "tiktonik_pose"

# Инициализируем единый инференсер с alias "human"
# (под капотом подтянет RTMDet для людей + 2D-позу; умеет видео/изображения;
# настраиваемые radius/thickness)
inferencer = MMPoseInferencer(
    "human",
    device=device,
)

# Готовим writer с кодеком H.264
vis_dir = out_dir / "visualization"
vis_dir.mkdir(parents=True, exist_ok=True)
out_video = vis_dir / (video_pth.stem + "_pose.mp4")

writer = imageio.get_writer(
    out_video.as_posix(),
    fps=fps,
    codec="libx264",
    format="FFMPEG",
    output_params=["-pix_fmt", "yuv420p"],
)

# Запускаем ленивый генератор инференса по видео.
# Чтобы получать кадры с отрисовкой в Python, включаем return_vis=True
# и задаем толщину/радиус. При желании можно включить/выключить рамки:
# draw_bbox=True/False.
result_gen = inferencer(
    str(video_pth),
    show=False,
    return_vis=True,
    radius=12,
    thickness=6,
    draw_bbox=False,
)

# Пробегаем все результаты и пишем в MP4
frames_written = 0
for res in result_gen:
    # 1) Заберём визуализацию (может быть списком или None)
    vis = res.get("visualization")
    if isinstance(vis, list):
        vis = vis[0] if vis else None

    # 2) Если отрисовка вернулась путём (бывает), читаем с диска
    if vis is None:
        vis_path = res.get("visualization_path")
        if isinstance(vis_path, list):
            vis_path = vis_path[0] if vis_path else None
        if vis_path:
            vis = imageio.v2.imread(vis_path)  # RGB

    if vis is None:
        continue  # нечего писать

    # 3) Приводим к RGB uint8 HxWx3
    frame = np.asarray(vis)
    if frame.ndim == 2:  # если ч/б
        frame = np.repeat(frame[..., None], 3, axis=2)
    elif (
        frame.ndim == 3 and frame.shape[2] == 4
    ):  # если RGBA — отбрасываем альфу
        frame = frame[..., :3]
    if frame.dtype != np.uint8:
        frame = np.clip(frame, 0, 255).astype(np.uint8)

    # 4) Подгоним размер под исходное видео (на всякий)
    if frame.shape[1] != w or frame.shape[0] != h:
        frame = cv2.resize(frame, (w, h), interpolation=cv2.INTER_LINEAR)

    # 5) Пишем в MP4 (imageio ожидает RGB)
    writer.append_data(frame)
    frames_written += 1
    if frames_written % 50 == 0:
        print(f"Обработано кадров: {frames_written}")

writer.close()
print(f"Готово! Сохранено кадров: {frames_written}")
print("Выходной файл:", out_video)

Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmpose-m_simcc-body7_pt-body7_420e-256x192-e48f03d0_20230504.pth
11/14 11:33:53 - mmengine - WARNING - Failed to search registry with scope "mmpose" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpose" is a correct scope, or whether the registry is initialized.
Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmdet_m_8xb32-100e_coco-obj365-person-235e8209.pth
11/14 11:33:53 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet" is a corre

/Users/pussykiller/Учеба/Семестр 3/Проектный практикум/CV-Workout-Tracker/venv311/lib/python3.11/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3550.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (482, 360) to (496, 368) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
[vost#0:0/libx264 @ 0x1229054c0] Multiple -pix_fmt options specified for stream 0, only the last option '-pix_fmt yuv420p' will be used.


Обработано кадров: 50
Обработано кадров: 100
Обработано кадров: 150
Обработано кадров: 200
Обработано кадров: 250
Обработано кадров: 300
Обработано кадров: 350
Обработано кадров: 400
Обработано кадров: 450
Обработано кадров: 500
Обработано кадров: 550
Обработано кадров: 600
Обработано кадров: 650
Обработано кадров: 700
Обработано кадров: 750
Обработано кадров: 800
Обработано кадров: 850
Обработано кадров: 900
Обработано кадров: 950
Обработано кадров: 1000
Обработано кадров: 1050
Обработано кадров: 1100
Обработано кадров: 1150
Готово! Сохранено кадров: 1185
Выходной файл: /Users/pussykiller/Учеба/Семестр 3/Проектный практикум/CV-Workout-Tracker/notebooks/mmpose/output_compare_angles/tiktonik_pose/visualization/tiktonik_360p_pose.mp4


In [ ]:
import math
import os
import random
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from kaggle_datasets import KaggleDatasets
from sklearn.metrics import f1_score
from tensorflow.keras import Model, layers, utils
from tensorflow.keras.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    ReduceLROnPlateau,
)
from tensorflow.keras.layers import (
    AveragePooling2D,
    BatchNormalization,
    Conv2D,
    Dense,
    Dropout,
    Flatten,
    GaussianDropout,
    GlobalAveragePooling2D,
    Input,
    LeakyReLU,
    MaxPooling2D,
    SpatialDropout2D,
)
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.regularizers import *
from tensorflow.random import set_seed


def seed_everything(seed):
    np.random.seed(seed)
    set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    os.environ["TF_DETERMINISTIC_OPS"] = "1"


SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

In [ ]:
AUTO = tf.data.AUTOTUNE

# GPU settings
gpus = tf.config.list_physical_devices("GPU")
print("GPUs:", gpus)
if gpus:
    try:
        for g in gpus:
            tf.config.experimental.set_memory_growth(g, True)
    except Exception as e:
        print("mem growth err:", e)

# mixed precision ускоряет на T4/P100
from tensorflow.keras import mixed_precision

mixed_precision.set_global_policy("mixed_float16")
print("policy:", mixed_precision.global_policy())

In [ ]:
GCS_DS_PATH = KaggleDatasets().get_gcs_path()

IMAGE_SIZE = [331, 331]  # ключевое отличие от 224
NUM_CLASSES = 104
EPOCHS_STAGE1 = 4
EPOCHS_STAGE2 = 12
EPOCHS_STAGE3 = 8

BATCH_SIZE = 16  # для 331 на T4 обычно норм; если OOM -> 8

GCS_PATH_SELECT = {
    192: GCS_DS_PATH + "/tfrecords-jpeg-192x192",
    224: GCS_DS_PATH + "/tfrecords-jpeg-224x224",
    331: GCS_DS_PATH + "/tfrecords-jpeg-331x331",
    512: GCS_DS_PATH + "/tfrecords-jpeg-512x512",
}
GCS_PATH = GCS_PATH_SELECT[IMAGE_SIZE[0]]

TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + "/train/*.tfrec")
VALIDATION_FILENAMES = tf.io.gfile.glob(GCS_PATH + "/val/*.tfrec")
TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + "/test/*.tfrec")


def count_samples(filenames):
    counts = [int(re.search(r"-([0-9]+)\.", f).group(1)) for f in filenames]
    return int(np.sum(counts))


NUM_TRAINING_IMAGES = count_samples(TRAINING_FILENAMES)
NUM_VALIDATION_IMAGES = count_samples(VALIDATION_FILENAMES)
NUM_TEST_IMAGES = count_samples(TEST_FILENAMES)

STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
VAL_STEPS = math.ceil(NUM_VALIDATION_IMAGES / BATCH_SIZE)

print(
    "train/val/test:",
    NUM_TRAINING_IMAGES,
    NUM_VALIDATION_IMAGES,
    NUM_TEST_IMAGES,
)
print("steps:", STEPS_PER_EPOCH, "val_steps:", VAL_STEPS)

In [ ]:
def decode_and_resize(image_data):
    img = tf.image.decode_jpeg(image_data, channels=3)
    img = tf.image.resize(img, IMAGE_SIZE, method="bilinear")
    img = tf.cast(img, tf.float32) / 255.0
    return img


def read_labeled(example):
    features = tf.io.parse_single_example(
        example,
        {
            "image": tf.io.FixedLenFeature([], tf.string),
            "class": tf.io.FixedLenFeature([], tf.int64),
        },
    )
    return decode_and_resize(features["image"]), tf.cast(
        features["class"], tf.int32
    )


def read_unlabeled(example):
    features = tf.io.parse_single_example(
        example,
        {
            "image": tf.io.FixedLenFeature([], tf.string),
            "id": tf.io.FixedLenFeature([], tf.string),
        },
    )
    return decode_and_resize(features["image"]), features["id"]


# Keras-aug слои (быстро на GPU)
augmenter = tf.keras.Sequential(
    [
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.10),
        tf.keras.layers.RandomZoom(0.10),
        tf.keras.layers.RandomTranslation(0.06, 0.06),
        tf.keras.layers.RandomContrast(0.15),
    ],
    name="augmenter",
)


def augment(image, label):
    image = augmenter(image, training=True)
    image = tf.clip_by_value(image, 0.0, 1.0)
    return image, label


def build_dataset(filenames, labeled=True, ordered=False, do_augment=False):
    opt = tf.data.Options()
    opt.experimental_deterministic = ordered
    ds = tf.data.TFRecordDataset(
        filenames, num_parallel_reads=AUTO
    ).with_options(opt)
    ds = ds.map(
        read_labeled if labeled else read_unlabeled, num_parallel_calls=AUTO
    )
    if labeled and do_augment:
        ds = ds.map(augment, num_parallel_calls=AUTO)
    return ds


def get_train_ds():
    ds = build_dataset(
        TRAINING_FILENAMES, labeled=True, ordered=False, do_augment=True
    )
    ds = ds.shuffle(2048, seed=SEED).repeat()
    ds = ds.batch(BATCH_SIZE, drop_remainder=True).prefetch(AUTO)
    return ds


def get_val_ds():
    ds = build_dataset(
        VALIDATION_FILENAMES, labeled=True, ordered=True, do_augment=False
    )
    ds = ds.batch(BATCH_SIZE, drop_remainder=False).cache().prefetch(AUTO)
    return ds


def get_test_ds():
    ds = build_dataset(
        TEST_FILENAMES, labeled=False, ordered=True, do_augment=False
    )
    ds = ds.batch(BATCH_SIZE, drop_remainder=False).prefetch(AUTO)
    return ds


val_ds = get_val_ds()

In [ ]:
class MacroF1(tf.keras.callbacks.Callback):
    def __init__(self, val_ds):
        super().__init__()
        self.val_ds = val_ds

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        y_true, y_pred = [], []
        for x, y in self.val_ds:
            p = self.model.predict_on_batch(x)
            y_pred.append(np.argmax(p, axis=1))
            y_true.append(y.numpy())
        y_true = np.concatenate(y_true)
        y_pred = np.concatenate(y_pred)
        score = f1_score(y_true, y_pred, average="macro")
        logs["val_macro_f1"] = score
        print(f"\nval_macro_f1: {score:.5f}")

In [ ]:
def build_model():
    inp = tf.keras.Input(shape=(*IMAGE_SIZE, 3))
    x = inp * 255.0
    x = tf.keras.applications.efficientnet.preprocess_input(x)

    base = tf.keras.applications.EfficientNetB3(
        include_top=False, weights="imagenet", input_shape=(*IMAGE_SIZE, 3)
    )
    base.trainable = False

    x = base(x, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.35)(x)
    out = tf.keras.layers.Dense(
        NUM_CLASSES, activation="softmax", dtype="float32"
    )(x)
    model = tf.keras.Model(inp, out)
    return model, base


def cosine_lr(max_lr, min_lr, total_steps):
    # простая cosine decay без warmup
    def schedule(step):
        t = tf.cast(step, tf.float32) / tf.cast(total_steps, tf.float32)
        lr = min_lr + 0.5 * (max_lr - min_lr) * (1 + tf.cos(np.pi * t))
        return lr

    return schedule


model, base = build_model()
model.summary()

In [ ]:
f1_cb = MacroF1(val_ds)

# Этап 1: учим только голову (быстро)
total_steps1 = STEPS_PER_EPOCH * EPOCHS_STAGE1
lr1 = tf.keras.optimizers.schedules.LearningRateSchedule(
    cosine_lr(1e-3, 2e-4, total_steps1)
)

opt1 = tf.keras.optimizers.Adam(learning_rate=lr1)
model.compile(
    optimizer=opt1,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(label_smoothing=0.1),
    metrics=["sparse_categorical_accuracy"],
)

cb1 = [
    f1_cb,
    tf.keras.callbacks.EarlyStopping(
        monitor="val_macro_f1",
        mode="max",
        patience=2,
        restore_best_weights=True,
    ),
]

hist1 = model.fit(
    get_train_ds(),
    steps_per_epoch=STEPS_PER_EPOCH,
    epochs=EPOCHS_STAGE1,
    validation_data=val_ds,
    callbacks=cb1,
    verbose=1,
)

# Этап 2: разморозить верхнюю часть backbone
base.trainable = True
for layer in base.layers[:-60]:
    layer.trainable = False

total_steps2 = STEPS_PER_EPOCH * EPOCHS_STAGE2
lr2 = tf.keras.optimizers.schedules.LearningRateSchedule(
    cosine_lr(3e-5, 6e-6, total_steps2)
)

# AdamW чуть лучше держит fine-tune
opt2 = tf.keras.optimizers.AdamW(learning_rate=lr2, weight_decay=1e-4)

model.compile(
    optimizer=opt2,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(label_smoothing=0.1),
    metrics=["sparse_categorical_accuracy"],
)

cb2 = [
    f1_cb,
    tf.keras.callbacks.EarlyStopping(
        monitor="val_macro_f1",
        mode="max",
        patience=4,
        restore_best_weights=True,
    ),
]

hist2 = model.fit(
    get_train_ds(),
    steps_per_epoch=STEPS_PER_EPOCH,
    epochs=EPOCHS_STAGE2,
    validation_data=val_ds,
    callbacks=cb2,
    verbose=1,
)

# Этап 3: можно ещё чуть разморозить (если val_macro_f1 < 0.955)
for layer in base.layers[:-20]:
    layer.trainable = False

total_steps3 = STEPS_PER_EPOCH * EPOCHS_STAGE3
lr3 = tf.keras.optimizers.schedules.LearningRateSchedule(
    cosine_lr(1e-5, 2e-6, total_steps3)
)
opt3 = tf.keras.optimizers.AdamW(learning_rate=lr3, weight_decay=1e-4)

model.compile(
    optimizer=opt3,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(label_smoothing=0.05),
    metrics=["sparse_categorical_accuracy"],
)

cb3 = [
    f1_cb,
    tf.keras.callbacks.EarlyStopping(
        monitor="val_macro_f1",
        mode="max",
        patience=4,
        restore_best_weights=True,
    ),
]

hist3 = model.fit(
    get_train_ds(),
    steps_per_epoch=STEPS_PER_EPOCH,
    epochs=EPOCHS_STAGE3,
    validation_data=val_ds,
    callbacks=cb3,
    verbose=1,
)

In [ ]:
test_ds = get_test_ds()

test_ids = []
preds = []

for x, ids in test_ds:
    # TTA: обычный + горизонтальный флип
    p1 = model.predict_on_batch(x)
    p2 = model.predict_on_batch(tf.image.flip_left_right(x))
    p = (p1 + p2) / 2.0

    preds.append(np.argmax(p, axis=1))
    test_ids.extend([i.decode("utf-8") for i in ids.numpy()])

preds = np.concatenate(preds)
sub = pd.DataFrame({"id": test_ids, "label": preds.astype(int)})
sub.to_csv("submission.csv", index=False)
sub.head()

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print("Running on TPU ", tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
GCS_DS_PATH = KaggleDatasets().get_gcs_path()

In [ ]:
IMAGE_SIZE = [
    224,
    224,
]
EPOCHS = 30
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
NUM_CLASSES = 104

GCS_PATH_SELECT = {  # available image sizes
    192: GCS_DS_PATH + "/tfrecords-jpeg-192x192",
    224: GCS_DS_PATH + "/tfrecords-jpeg-224x224",
    331: GCS_DS_PATH + "/tfrecords-jpeg-331x331",
    512: GCS_DS_PATH + "/tfrecords-jpeg-512x512",
}
GCS_PATH = GCS_PATH_SELECT[IMAGE_SIZE[0]]

TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + "/train/*.tfrec")
VALIDATION_FILENAMES = tf.io.gfile.glob(GCS_PATH + "/val/*.tfrec")
TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + "/test/*.tfrec")

AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
# 1. Декодирование и обработка изображений


def decode_and_resize(image_data):
    """
    Декодирует JPEG-изображение и приводит его к заданному размеру.
    Нормализует пиксели в диапазон [0, 1].
    """
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.image.resize(image, IMAGE_SIZE, method="bilinear")
    image = tf.cast(image, tf.float32) / 255.0
    return image


def augment_image(image, label):
    """
    Применяет лёгкую аугментацию только к обучающим изображениям.
    Помогает улучшить обобщающую способность модели.
    """
    # Случайное горизонтальное отражение
    image = tf.image.random_flip_left_right(image)
    # Случайные изменения яркости и насыщенности
    image = tf.image.random_brightness(image, max_delta=0.15)
    image = tf.image.random_saturation(image, lower=0.8, upper=1.2)
    return image, label

In [ ]:
# 2. Чтение данных из TFRecord


def read_labeled_record(example):
    """Парсит пример с меткой класса (для train/val)."""
    features = tf.io.parse_single_example(
        example,
        {
            "image": tf.io.FixedLenFeature([], tf.string),
            "class": tf.io.FixedLenFeature([], tf.int64),
        },
    )
    image = decode_and_resize(features["image"])
    label = tf.cast(features["class"], tf.int32)
    return image, label


def read_unlabeled_record(example):
    """Парсит тестовый пример без метки — только id."""
    features = tf.io.parse_single_example(
        example,
        {
            "image": tf.io.FixedLenFeature([], tf.string),
            "id": tf.io.FixedLenFeature([], tf.string),
        },
    )
    image = decode_and_resize(features["image"])
    return image, features["id"]

In [ ]:
# 3. Формирование датасетов


def build_dataset(filenames, labeled=True, ordered=False, augment=False):
    """
    Создаёт tf.data.Dataset из списка TFRecord-файлов.
    Параметры:
        labeled   — есть ли метки (train/val vs test)
        ordered   — сохранять ли порядок (важно для сабмита)
        augment   — применять ли аугментацию (только для train)
    """
    options = tf.data.Options()
    if not ordered:
        options.experimental_deterministic = False  # ускоряет чтение

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
    dataset = dataset.with_options(options)

    parse_fn = read_labeled_record if labeled else read_unlabeled_record
    dataset = dataset.map(parse_fn, num_parallel_calls=AUTO)

    if augment:
        dataset = dataset.map(augment_image, num_parallel_calls=AUTO)

    return dataset


def get_training_dataset():
    """Обучающий датасет с аугментацией, перемешиванием и повторением."""
    dataset = build_dataset(TRAINING_FILENAMES, labeled=True, augment=True)
    dataset = dataset.repeat()
    dataset = dataset.shuffle(buffer_size=2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset


def get_validation_dataset():
    """Валидационный датасет — без аугментации, с кэшированием."""
    dataset = build_dataset(VALIDATION_FILENAMES, labeled=True, ordered=False)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()  # ускоряет валидацию при повторных эпохах
    dataset = dataset.prefetch(AUTO)
    return dataset


def get_test_dataset(ordered=True):
    """Тестовый датасет — порядок важен для корректного сабмита."""
    dataset = build_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

In [ ]:
# 4. Вспомогательные функции


def count_samples(filenames):
    """Подсчитывает общее число примеров по именам TFRecord-файлов."""
    counts = [int(re.search(r"-([0-9]+)\.", f).group(1)) for f in filenames]
    return np.sum(counts)


# Расчёт количества примеров и шагов на эпоху
NUM_TRAINING_IMAGES = count_samples(TRAINING_FILENAMES)
NUM_VALIDATION_IMAGES = count_samples(VALIDATION_FILENAMES)
NUM_TEST_IMAGES = count_samples(TEST_FILENAMES)
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE

print(
    f"Загружено данных: "
    f"{NUM_TRAINING_IMAGES} обучающих, "
    f"{NUM_VALIDATION_IMAGES} валидационных, "
    f"{NUM_TEST_IMAGES} тестовых изображений."
)

In [ ]:
def basic_block(x, filters, stride=1):
    """
    Базовый ResNet-блок (как в ResNet-18/34).
    """
    shortcut = x

    # Основной путь
    x = layers.Conv2D(
        filters, 3, strides=stride, padding="same", use_bias=False
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(filters, 3, strides=1, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)

    # Skip-соединение: согласуем размерность, если нужно
    if stride != 1 or shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, 1, strides=stride, use_bias=False)(
            shortcut
        )
        shortcut = layers.BatchNormalization()(shortcut)

    # Сложение и активация
    x = layers.Add()([x, shortcut])
    x = layers.ReLU()(x)
    return x


def build_resnet34(input_shape=(331, 331, 3), num_classes=104):
    """
    Создаёт ResNet-34-подобную сеть.
    """
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(64, 7, strides=2, padding="same", use_bias=False)(
        inputs
    )  # 331 → 166
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPool2D(3, strides=2, padding="same")(x)  # 166 → 83

    # Стадии ResNet: [3, 4, 6, 3] блоков
    # Stage 1: 64 filters, stride=1 → размер не меняется (83)
    for _ in range(3):
        x = basic_block(x, 64, stride=1)

    # Stage 2: 128 filters, первый блок с stride=2 → 83 → 42
    x = basic_block(x, 128, stride=2)
    for _ in range(3):
        x = basic_block(x, 128, stride=1)

    # Stage 3: 256 filters, первый блок с stride=2 → 42 → 21
    x = basic_block(x, 256, stride=2)
    for _ in range(5):
        x = basic_block(x, 256, stride=1)

    # Stage 4: 512 filters, первый блок с stride=2 → 21 → 11
    x = basic_block(x, 512, stride=2)
    for _ in range(2):
        x = basic_block(x, 512, stride=1)

    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(
        num_classes, activation="softmax", name="predictions"
    )(x)

    return Model(inputs, outputs, name="ResNet34_Custom")

In [ ]:
model = build_resnet34(input_shape=(*IMAGE_SIZE, 3), num_classes=NUM_CLASSES)

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

# Вывод информации о модели
model.summary()

In [ ]:
def get_model():
    return build_resnet34(
        input_shape=(*IMAGE_SIZE, 3), num_classes=NUM_CLASSES
    )


callbacks_list = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=6, restore_best_weights=True, verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=3, verbose=1
    ),
]

print("Начинаем обучение модели...")
print(f"  * Общее число эпох: {EPOCHS}")
print(
    f"  * Batch size: {BATCH_SIZE} (реплик: {strategy.num_replicas_in_sync})"
)
print(f"  * Шагов на эпоху: {STEPS_PER_EPOCH}")
print(f"  * Обучающих изображений: {NUM_TRAINING_IMAGES}")
print(f"  * Валидационных изображений: {NUM_VALIDATION_IMAGES}")
print("-" * 50)

# --- Создание и компиляция модели ---
with strategy.scope():
    model = get_model()
    model.compile(
        optimizer="nadam",
        loss="sparse_categorical_crossentropy",
        metrics=["sparse_categorical_accuracy"],
    )

# --- Запуск обучения ---
history = model.fit(
    get_training_dataset(),
    steps_per_epoch=STEPS_PER_EPOCH,
    epochs=EPOCHS,
    validation_data=get_validation_dataset(),
    callbacks=callbacks_list,
    verbose=1,  # вывод по эпохам
)

print("Обучение завершено!")
print(f"  * Использовано эпох: {len(history.history['loss'])}")
if "lr" in history.history:
    print(f"  * Финальная learning rate: {history.history['lr'][-1]:.2e}")